### To-Do
- csv file c1(csv file id) and c2 (meta file location)
- extract meta-data from file
- enriched csv file with metadata
- create EML file from metadata xml file

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_colwidth=0

In [4]:
def txt_csv(txt_file):
    file_prefix = txt_file.split('.txt')[0]
    print(file_prefix)
    df = pd.read_csv(txt_file, header=None, names=['Path'])
    df['Path'] = df['Path'].str.replace('/content/drive/MyDrive/Digispecies/Data', './local_data/metadata')
    df.to_csv(file_prefix+'.csv', index=True, index_label='Image_Id')
    return None

txt_files = ['./local_data/table_list/02_dec.txt', './local_data/table_list/05_dec.txt', 
            './local_data/table_list/06_jul.txt']

for file in txt_files:
    txt_csv(file)

./local_data/table_list/02_dec
./local_data/table_list/05_dec
./local_data/table_list/06_jul


In [ ]:
# get all metafiles from google drive data directory
import os
from glob import glob
import shutil
from pathlib import Path

# Specify the root directory path
# MyDrive/Work/Digi Species/Data/

def get_all_meta_files(PATH, META_XML="meta.xml", EXT="*.xml"):
  # Get a list of all CSV files in the directory and its subdirectories
  EXT="*.xml"
  all_metaxml_files = [file for path, _, files in os.walk(PATH)\
                       for file in glob(os.path.join(path, EXT)) if 'meta.xml' in file]
  # all_metaxml_files = [file for path, _, files in os.walk(PATH) for file in os.path.join(path, FILE)]
  
  # print(all_metaxml_files)

  for file_src in all_metaxml_files:
    file_dst = file_src.replace("/content/drive/MyDrive/Work/Digi Species/Data/Data/", \
                                "/content/drive/MyDrive/Work/Digi Species/Data/Meta_Data/")
    # Create a Path object from the file path
    path = Path(file_dst)

    # Create the parent directories if they don't exist
    path.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(file_src, path)


PATH_list = ["/content/drive/MyDrive/Work/Digi Species/Data/Data/unziped_02_dec/",
            "/content/drive/MyDrive/Work/Digi Species/Data/Data/unziped_05_dec/",
             "/content/drive/MyDrive/Work/Digi Species/Data/Data/Only_table_06_jul/"]

for PATH in PATH_list:
  get_all_meta_files(PATH)

In [6]:
def extract_metadata(xml_file):
    pass
    

def read_path(csv_file):
    df = pd.read_csv(csv_file, encoding='utf-8')
    return df

def set_meta_file_path(path):
    path = path.split('images/')[0]
    return path+'meta.xml'
    


In [7]:
from IPython.display import display, HTML

"""('AdditionalNotes', 'Sonderabdruck aus, Decheniana, Verhandlungen des Naturhistorischen Vereins der Rheinlande und Westfalens, Band 101 A B'),
   'PlaceOfPublication', 'Bonn',
   'PublisherName', 'Hch. Ludwig',
   'TxMainLanguage', 'german',
   'TxType', 'book',
   'TxSpeciesData', 'yes',
   'TxFormat', 'A4',
   'SizeSourcePrint', 'Seiten 131-214g, L1-L18',
   'Illustration', 'Diagramme',
   'pathimagefiles', './images/',
   '_representative', '1',
   'Author', {'type', 'person',
             'lastName', 'Budde',
             'firstName', 'Hermann',
             'authorityID', 'gnd',
             'authorityURI', 'http,//d-nb.info/gnd/',
             'authorityValue', 1028522606,
             'displayName', 'Budde, Hermann'}}"""


mappping_metaxml_to_eml_list = [('PublicationYear', 'eventDate'), 
                                 ('PublicationYear', 'year'), 
                                  ('PublicationYear', 'publicationYear'),
                                 ('TitleDocMain', 'title'),('TitleDocMain', 'publicationTitle'), 
                                  ('TxID','catalogNumber'), ('DocLanguage', 'language'),
                                  ('singleDigCollection', 'collectionCode'), ('TxAuthor','publicationAuthors'),
                                  ('singleDigCollection', 'parentCollectionIdentifier'), 
                                 ('CatalogIDDigital', 'collectionIdentifier'), ('authorityURI', 'authorityURI'), 
                                  ('authorityValue', 'authorityValue'),]

mappping_metaxml_to_eml_dict = {i[1]:i[0] for i in mappping_metaxml_to_eml_list}

print (mappping_metaxml_to_eml_dict)

def remove_null_values_from_nested_dict(nested_dict):
    if not isinstance(nested_dict, dict):
        return nested_dict

    new_dict = {}
    for key, value in nested_dict.items():
        if value is not None and not isinstance(value, dict) and not value!=value:
            if type(value)==float:
                value = int(value)
            new_dict[key] = value
        elif isinstance(value, dict):
            if type(value)==float:
                value = int(value)
            new_dict[key] = remove_null_values_from_nested_dict(value)

    return new_dict



# Getting metadata from all metafiles for imageId_metapath file
def get_metadata_file(metafile_path_data_dict, imageId_metapath_file=None):
    for meta_file_key, value in metafile_path_data_dict.items():
        # ToDo -------- Temporary Path -------------
        # download data and append ./ to meta_file_key
        # meta_file_key ='./local_data/metadata/meta.xml' # todo metafile 
        df_meta_file = pd.read_xml(path_or_buffer=meta_file_key, xpath="//goobi:metadata",
                          namespaces={"goobi":"http://meta.goobi.org/v1.5.1/"})
        df_meta_file.set_index(keys='name', inplace=True)
        df_meta_file = df_meta_file[~df_meta_file.index.duplicated()]
        dict_metadata = df_meta_file.to_dict()
        new_dict = {}
        new_dict['metadata'] = remove_null_values_from_nested_dict(df_meta_file['metadata'].to_dict())
        new_dict['Author'] = remove_null_values_from_nested_dict(df_meta_file.loc['Author'].to_dict())
        # metafile_path_data_dict[key]=new_dict
    
        new_dict1 = {}
        for k, v in new_dict.items():
            # print (new_dict)
            k1_v1_dict = {k1:v1 for k1, v1 in new_dict[k].items() if k1 in mappping_metaxml_to_eml_dict.values()}
            # print(k1_v1_dict)
            new_dict1.update({k2:k1_v1_dict[k1] for k2, k1 in mappping_metaxml_to_eml_dict.items() if k1 in k1_v1_dict.keys()})
        metafile_path_data_dict[meta_file_key] = new_dict1
        # Todo -------- Remove Break ----------------
        # break
    
    
    df_meta = pd.DataFrame.from_dict(metafile_path_data_dict, orient='index' ).reset_index()
    # display(df_meta)
    ## temporary testing
    #  this line is not needed if you have metadata xml files
    # df_meta.loc[0,'index']='unziped_02_dec/tx_7487/29001/meta.xml' 
    # display(df_meta)
    # print(df_meta)
    df = df_imageId_metapath.merge(df_meta, left_on='MetaFile', right_on='index', how='left')
    imageId_metapath_file_prefix = imageId_metapath_file.split('.csv')[0]
    df.to_csv(imageId_metapath_file_prefix+'_metadata.csv', encoding='Utf', index=False)


imageId_metapath_file_list = ['./local_data/table_list/02_dec.csv', 
                         './local_data/table_list/05_dec.csv',
                         './local_data/table_list/06_jul.csv']
for imageId_metapath_file in imageId_metapath_file_list:
    df_imageId_metapath = read_path(imageId_metapath_file)
    df_imageId_metapath['MetaFile'] = df_imageId_metapath['Path'].apply(set_meta_file_path)
    df_imageId_metapath.sort_values(by='MetaFile',inplace=True)
    metafile_path_data_dict = {i:{} for i in df_imageId_metapath['MetaFile'].unique().tolist()}
    print(metafile_path_data_dict)
    get_metadata_file(metafile_path_data_dict, imageId_metapath_file)

{'eventDate': 'PublicationYear', 'year': 'PublicationYear', 'publicationYear': 'PublicationYear', 'title': 'TitleDocMain', 'publicationTitle': 'TitleDocMain', 'catalogNumber': 'TxID', 'language': 'DocLanguage', 'collectionCode': 'singleDigCollection', 'publicationAuthors': 'TxAuthor', 'parentCollectionIdentifier': 'singleDigCollection', 'collectionIdentifier': 'CatalogIDDigital', 'authorityURI': 'authorityURI', 'authorityValue': 'authorityValue'}
{'./local_data/metadata/unziped_02_dec/tx_7397/28978/meta.xml': {}, './local_data/metadata/unziped_02_dec/tx_7407/28981/meta.xml': {}, './local_data/metadata/unziped_02_dec/tx_7423/28986/meta.xml': {}, './local_data/metadata/unziped_02_dec/tx_7426/28987/meta.xml': {}, './local_data/metadata/unziped_02_dec/tx_7447/28988/meta.xml': {}, './local_data/metadata/unziped_02_dec/tx_7454/28990/meta.xml': {}, './local_data/metadata/unziped_02_dec/tx_7462/28991/meta.xml': {}, './local_data/metadata/unziped_02_dec/tx_7464/28992/meta.xml': {}, './local_dat